Anvil code which is connected with Faster R-CNN training model

In [ ]:
from ._anvil_designer import Form1Template
from anvil import *
import anvil.server
import datetime

class Form1(Form1Template):
    def __init__(self, **properties):
        self.init_components(**properties)
        self.label_dominant_color.visible = False
        self.label_dish_suggestion.visible = False
        self.label_olive_message.visible = False
        self.label_timing.visible = False
        self.label_speed.visible = False
        self.button_yes.visible = False
        self.button_no.visible = False
        self.label_prompt.visible = False
        self.label_olive_name.visible = False
        self.button_generate_report.visible = False
        
        # Link button events to their handlers
        self.button_yes.set_event_handler('click', self.button_yes_click)
        self.button_no.set_event_handler('click', self.button_no_click)
        self.button_generate_report.set_event_handler('click', self.generate_report)

    def file_loader_1_change(self, **event_args):
        uploaded_file = self.file_loader_1.file
        if uploaded_file:
            self.image_display.source = uploaded_file
            self.label_prompt.text = "Do you want to analyze the image and see the results?"
            self.label_prompt.visible = True
            self.button_yes.visible = True
            self.button_no.visible = True
            self.label_dominant_color.visible = False
            self.label_dish_suggestion.visible = False
            self.label_olive_message.visible = False
            self.label_timing.visible = False
            self.label_speed.visible = False
            self.label_olive_name.visible = False
            self.button_generate_report.visible = False

    def button_yes_click(self, **event_args):
        uploaded_file = self.file_loader_1.file
        if uploaded_file:
            self.process_and_display_results(uploaded_file)

    def button_no_click(self, **event_args):
        self.file_loader_1.clear()
        self.reset_display()
              
    def process_and_display_results(self, uploaded_file):
        try:
            result = anvil.server.call('classify_image', uploaded_file)
            print(f"Result received: {result}")

            if "error" in result:
                self.label_olive_message.text = result["error"]
                self.label_olive_message.visible = True
                return

            hex_color = result.get('dominant_color', 'N/A')
            color_name = result.get('color_name', 'N/A')
            olive_color_message = result.get('olive_color_message', '')
            preservation_tips = result.get('preservation_tips', 'N/A')
            health_tips = result.get('health_tips', 'N/A')
            timing_message = result.get('timing_message', 'Timing info not available')
            speed_message = result.get('speed_message', 'Speed info not available')
            dish_suggestion = result.get('dish_suggestion', 'N/A')
            olive_name = result.get('olive_name', 'N/A')
            
            self.label_olive_name.text = f"Olive Name: {olive_name}"
            self.label_dominant_color.text = f"Dominant Color: {hex_color} ({color_name})"
            self.label_olive_message.text = f"{olive_color_message}\n\nPreservation Tips: {preservation_tips}\n\nHealth Tips: {health_tips}"
            self.label_timing.text = timing_message
            self.label_speed.text = speed_message
            self.label_dish_suggestion.text = f"Suggested Dish: {dish_suggestion}"
            
            self.label_olive_name.visible = True
            self.label_dominant_color.visible = True
            self.label_dish_suggestion.visible = True
            self.label_olive_message.visible = True
            self.label_timing.visible = True
            self.label_speed.visible = True
            self.button_generate_report.visible = True
            
            ripeness_message = self.get_ripeness_notification(olive_color_message)
            if ripeness_message:
                self.show_alert_notification(ripeness_message)

            self.result_for_report = result

        except Exception as e:
            self.label_olive_message.text = f"Error processing the image: {str(e)}"
            self.label_olive_message.visible = True
            
        self.button_yes.visible = False
        self.button_no.visible = False
        self.label_prompt.visible = False

    def get_ripeness_notification(self, olive_color_message):
        if 'unripe' in olive_color_message.lower():
            return "The olives are unripe. Preferably, they should be classified based on their ripeness for olive oil."
        elif 'partially ripe' in olive_color_message.lower():
            return "The olives are partially ripe. Preferably, they should be classified based on their ripeness for olive oil."
        elif 'fully ripe' in olive_color_message.lower():
            return "The olives are fully ripe. Preferably, they should be classified based on their ripeness for olive oil."
        elif 'overripe' in olive_color_message.lower():
            return "The olives are overripe. Preferably, they should be classified based on their ripeness for olive oil."
        else:
            return None

    def show_alert_notification(self, message):
        alert(message)

    def generate_report(self, **event_args):
        if hasattr(self, 'result_for_report'):
            result = self.result_for_report
            current_date = datetime.datetime.now().strftime('%A, %B %d, %Y')
            current_time = datetime.datetime.now().strftime('%H:%M:%S')
            pdf_report = anvil.server.call(
                'generate_report',
                result['olive_name'],
                result['dominant_color'],
                result['color_name'],
                result['olive_color_message'],
                result['preservation_tips'],
                result['health_tips'],
                result['dish_suggestion'],
                current_date,
                current_time
            )
            download(pdf_report)

    def reset_display(self):
        self.button_yes.visible = False
        self.button_no.visible = False
        self.label_olive_name.visible = False
        self.label_dominant_color.visible = False
        self.label_dish_suggestion.visible = False
        self.label_olive_message.visible = False
        self.label_timing.visible = False
        self.label_speed.visible = False
        self.label_prompt.visible = False
        self.button_generate_report.visible = False
